In this lab, you should try to implement some of the techniques discussed in the lecture.
Here is a list of reasonable tasks.
 
Easy:
 * L1 or L2 regularization (choose one)
 * momentum, Nesterov's momentum (choose one)

Medium difficulty:
 * Adagrad, RMSProp (choose one)
 * dropout
 * data augmentation (tiny rotatations, up/down-scalings etc.)

Try to test your network to see if these changes improve accuracy. They improve accuracy much more if you increase the layer size, and if you add more layers.

In [2]:
import random
import numpy as np
from torchvision import datasets, transforms
from scipy.ndimage.interpolation import rotate
from scipy.ndimage import shift, zoom
import matplotlib.pyplot as plt


In [4]:
# Let's read the mnist dataset

def load_mnist(path='.'):
    train_set = datasets.MNIST(path, train=True, download=True)
    x_train = train_set.data.numpy()
    _y_train = train_set.targets.numpy()
    
    test_set = datasets.MNIST(path, train=False, download=True)
    x_test = test_set.data.numpy()
    _y_test = test_set.targets.numpy()
    
    x_train = x_train.reshape((x_train.shape[0],28*28)) / 255.
    x_test = x_test.reshape((x_test.shape[0],28*28)) / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1
    
    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    x_validation = x_train[:10000]
    y_validation = y_train[:10000]

    return (x_train[10000:], y_train[10000:]), (x_test, y_test), (x_validation, y_validation)

(x_train, y_train), (x_test, y_test), (x_validation, y_validation) = load_mnist()

In [5]:
class Augmenter:
    rotation: int
    horizontal_shift: int
    vertical_shift: int

    def __init__(self, rotation: int = 10, horizontal_shift: int = 4, vertical_shift: int = 2):
        self.rotation = rotation
        self.horizontal_shift = horizontal_shift
        self.vertical_shift = vertical_shift

    def rotate(self, a: np.array):
        rotated = rotate(a, np.random.normal(scale = self.rotation))
        while rotated.shape[0] != 28:
            rotated = np.delete(rotated, 0, axis = 0)
            if rotated.shape[0] != 28:
                rotated = np.delete(rotated, -1, axis = 0)

        while rotated.shape[1] != 28:
            rotated = np.delete(rotated, 0, axis = 1)
            if rotated.shape[1] != 28:
                rotated = np.delete(rotated, -1, axis = 1)

        return rotated
        
    def shift_horizontally(self, a: np.array):
        return shift(a, (0, np.random.normal(scale = self.horizontal_shift)), output = a)

    def shift_vertically(self, a: np.array):
        return shift(a, (np.random.normal(scale = self.vertical_shift), 0), output = a)\

    def augment(self, X: np.array, Y: np.array, n: int = 3):
        result_X = []
        result_Y = [] 

        for x, y in zip(X, Y):
            result_X.append(x)
            result_Y.append(y)

            for i in range(n):
                result_X.append(np.reshape(self.shift_horizontally(self.shift_vertically(self.rotate(np.reshape(x, (28, 28))))), 28*28))
                result_Y.append(y)

        return np.array(result_X), np.array(result_Y)

In [6]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

In [12]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.biases_accumulator = [np.zeros([y, 1]) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.weights_accumulator = [np.zeros([y, x]) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.matmul(w, a)+b)
        return a
    
    def update_mini_batch(self, mini_batch, eta, momentum=0, l2=0):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch which is as in tensorflow API.
        # eta is the learning rate      
        nabla_b, nabla_w = self.backprop(mini_batch[0].T,mini_batch[1].T)
        
        self.weights_accumulator = [momentum*w-(eta/len(mini_batch[0]))*nw 
                                    for w, nw in zip(self.weights_accumulator, nabla_w)]            
        self.weights = [(1.0 - l2)*w + wa for w, wa in zip(self.weights, self.weights_accumulator)]
        
        self.biases_accumulator = [momentum*b-(eta/len(mini_batch[0]))*nb 
                       for b, nb in zip(self.biases_accumulator, nabla_b)]
        self.biases = [b+ba for b, ba in zip(self.biases, self.biases_accumulator)]
        
    def backprop(self, x, y):
        # For a single input (x,y) return a pair of lists.
        # First contains gradients over biases, second over weights.
        g = x
        gs = [g] # list to store all the gs, layer by layer
        fs = [] # list to store all the fs, layer by layer
        for b, w in zip(self.biases, self.weights):
            f = np.dot(w, g)+b
            fs.append(f)
            g = sigmoid(f)
            gs.append(g)
        # backward pass <- both steps at once
        dLdg = self.cost_derivative(gs[-1], y)
        dLdfs = []
        for w,g in reversed(list(zip(self.weights,gs[1:]))):
            dLdf = np.multiply(dLdg,np.multiply(g,1-g))
            dLdfs.append(dLdf)
            dLdg = np.matmul(w.T, dLdf)
        
        dLdWs = [np.matmul(dLdf,g.T) for dLdf,g in zip(reversed(dLdfs),gs[:-1])] # automatic here
        dLdBs = [np.sum(dLdf,axis=1).reshape(dLdf.shape[0],1) for dLdf in reversed(dLdfs)] # CHANGE: Need to sum here
        return (dLdBs,dLdWs)

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, momentum=0, l2=0, early_stop=None, validation_data=None):
        x_train, y_train = training_data
        accuracy_val = []
        if validation_data:
            x_validation, y_validation = validation_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch((x_mini_batch, y_mini_batch), eta, momentum, l2)
            if validation_data:
                current_accuracy = self.evaluate((x_validation, y_validation))
                print("Epoch: {0}, Accuracy: {1}".format(j, current_accuracy))
                accuracy_val.append(current_accuracy)
                if early_stop and len(accuracy_val) > early_stop:
                    best_accuracy = max(accuracy_val[:-early_stop])
                    last_accuracy = max(accuracy_val[-early_stop:])
                    if last_accuracy <= best_accuracy:
                        print("Early stop")
                        break
            else:
                print("Epoch: {0}".format(j))
        return accuracy_val


In [10]:
network = Network([784,128,64,10])
augmenter = Augmenter()
train_data = augmenter.augment(x_train, y_train) 

In [18]:
accuracy_val = network.SGD(train_data, epochs=100, mini_batch_size=100, eta=0.03, l2=0.00001,
                       momentum=0.9, early_stop=10, validation_data=(x_validation, y_validation))

Epoch: 0, Accuracy: 0.8053
Epoch: 1, Accuracy: 0.8625
Epoch: 2, Accuracy: 0.9003
Epoch: 3, Accuracy: 0.919
Epoch: 4, Accuracy: 0.9311
Epoch: 5, Accuracy: 0.9392
Epoch: 6, Accuracy: 0.9453
Epoch: 7, Accuracy: 0.9494
Epoch: 8, Accuracy: 0.9529
Epoch: 9, Accuracy: 0.9548
Epoch: 10, Accuracy: 0.9565
Epoch: 11, Accuracy: 0.9588
Epoch: 12, Accuracy: 0.9604
Epoch: 13, Accuracy: 0.9609
Epoch: 14, Accuracy: 0.9624
Epoch: 15, Accuracy: 0.9635
Epoch: 16, Accuracy: 0.9637
Epoch: 17, Accuracy: 0.964
Epoch: 18, Accuracy: 0.9645
Epoch: 19, Accuracy: 0.9648
Epoch: 20, Accuracy: 0.9653
Epoch: 21, Accuracy: 0.9656
Epoch: 22, Accuracy: 0.9661
Epoch: 23, Accuracy: 0.9665
Epoch: 24, Accuracy: 0.9669
Epoch: 25, Accuracy: 0.9672
Epoch: 26, Accuracy: 0.9679
Epoch: 27, Accuracy: 0.9681
Epoch: 28, Accuracy: 0.9689
Epoch: 29, Accuracy: 0.9691
Epoch: 30, Accuracy: 0.9692
Epoch: 31, Accuracy: 0.9693
Epoch: 32, Accuracy: 0.9694
Epoch: 33, Accuracy: 0.9694
Epoch: 34, Accuracy: 0.9699
Epoch: 35, Accuracy: 0.9705
Epoc

In [19]:
final_accuracy = network.evaluate((x_test, y_test))
print("Accuracy on a test set:", final_accuracy)

Accuracy on a test set: 0.9763
